In [8]:
import os
import requests
import base64
import json

GPT4V_KEY = "[YOUr KEY]"  # please rotate the old one in Azure


AZURE_ENDPOINT = "https://first-azureopenai-11162025.openai.azure.com"
DEPLOYMENT_NAME = "gpt-4o"
API_VERSION = "2025-01-01-preview"

# Explicit list of invoice files in Lakehouse Files/raw_docs
IMAGE_PATH = "/lakehouse/default/Files/rawdocs/invoice_1.jpg"

encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that helps people find information."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_image}"
          }
        },
        {
          "type": "text",
          "text": "can you please explin the image?"
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

GPT4V_ENDPOINT = "https://first-azureopenai-11162025.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview"



# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
result = response.json()

# Assuming your JSON response is stored in a variable named `response_json`
response_json = result
# Accessing specific values
choices = response_json['choices']
message_content = choices[0]['message']['content']

# Print the content of the message
print(message_content)


StatementMeta(, 77a5e866-27b6-4648-ac71-2abd719bbb2e, 10, Finished, Available, Finished)

Certainly! The image is an **invoice** issued by a company named **Clearwave Logistics**. Below is a breakdown of the key details on the invoice:

---

### **Header Details**
- **Company Name**: Clearwave Logistics
- **Invoice Number**: #83457
- **Invoice Date**: September 6
- **Due Date**: September 20, 2023 (though "203" appears to be a typo, likely intended to be 2023).

---

### **Billing and Shipping Information**
1. **BILL TO**:
   - Name: **Everfresh Markets**
   - Address: 847 Elm Avenue, Columbus, OH 43201.

2. **SHIP TO**:
   - Name: **Greenway Foods**
   - Address: 3425 Pine Lane, Lexington, KY 40503.

---

### **Itemized Charges**
The table provides a breakdown of services, quantities, unit prices, and amounts:

1. **Freight Services**:
   - Quantity: 20
   - Unit Price: $150.00
   - Total Amount: $3,000.00

2. **Warehousing Fees**:
   - Quantity: 8
   - Unit Price: $80.00
   - Total Amount: $640.00

3. **Handling Charges**:
   - Quantity: 15
   - Unit Price: $20.00
   - To

In [13]:
import os
import base64
import requests
import json

GPT4V_KEY = "5BrxnGIZq8tfDFroDL477sxZd3fk4Aq5ImQ1JjpRD6cHpjTp46SlJQQJ99BKACYeBjFXJ3w3AAABACOGuJxx"  # please rotate the old one in Azure




AZURE_ENDPOINT = "https://first-azureopenai-11162025.openai.azure.com"
DEPLOYMENT_NAME = "gpt-4o"
API_VERSION = "2025-01-01-preview"


GPT4V_ENDPOINT = f"{AZURE_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"

FOLDER_PATH = "/lakehouse/default/Files/rawdocs/"
IMAGE_EXTS  = {".jpg", ".jpeg", ".png", ".pdf"}


headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

def call_vision_single(image_path: str):
    with open(image_path, "rb") as f:
        encoded_image = base64.b64encode(f.read()).decode("ascii")

    payload = {
      "messages": [
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": "You are an AI assistant that helps people find information."
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}"
              }
            },
            {
              "type": "text",
              "text": "Can you please explain the image?"
            }
          ]
        }
      ],
      "temperature": 0.7,
      "top_p": 0.95,
      "max_tokens": 200
    }

    resp = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    print(f"[DEBUG] {image_path} status:", resp.status_code, "len:", len(resp.content))

    if not resp.text or not resp.text.strip():
        print(f"❌ Empty body for {image_path}")
        return None

    result = resp.json()
    return result["choices"][0]["message"]["content"]

results = []

for filename in os.listdir(FOLDER_PATH):
    full_path = os.path.join(FOLDER_PATH, filename)

    if not os.path.isfile(full_path):
        continue

    _, ext = os.path.splitext(filename)
    if ext.lower() not in IMAGE_EXTS:
        print(f"Skipping non-image file: {filename}")
        continue

    print(f"🔎 Processing {filename} ...")
    explanation = call_vision_single(full_path)

    if explanation:
        results.append({
            "file": filename,
            "path": full_path,
            "explanation": explanation
        })
        print(f"✅ Done: {filename}")
    else:
        print(f"⚠️ No result for {filename}")


StatementMeta(, 77a5e866-27b6-4648-ac71-2abd719bbb2e, 15, Finished, Available, Finished)

🔎 Processing invoice_1.jpg ...
[DEBUG] /lakehouse/default/Files/rawdocs/invoice_1.jpg status: 200 len: 2045
✅ Done: invoice_1.jpg


In [1]:
# ============================================================
# IGNITE DEMO: INVOICE PIPELINE
# - AI (Vision) pipeline for images  -> invoices_ai
# - CSV pipeline for structured data -> invoices_csv
# ============================================================

import os
import base64
import json
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark: SparkSession = spark  


# ============================================================
# 1. Azure OpenAI Config
# ============================================================
AZURE_ENDPOINT  = "https://first-azureopenai-11162025.openai.azure.com"
DEPLOYMENT_NAME = "gpt-4o"
API_VERSION     = "2025-01-01-preview"
# ⚠️ For demos only – use Key Vault / secrets in real setup
GPT4V_KEY = "5BrxnGIZq8tfDFroDL477sxZd3fk4Aq5ImQ1JjpRD6cHpjTp46SlJQQJ99BKACYeBjFXJ3w3AAABACOGuJxx"  # please rotate the old one in Azure

GPT4V_ENDPOINT = (
    f"{AZURE_ENDPOINT}/openai/deployments/"
    f"{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
)

headers = {"Content-Type": "application/json", "api-key": GPT4V_KEY}


# ============================================================
# 2. Paths & Schema
# ============================================================
FOLDER_PATH = "/lakehouse/default/Files/rawdocs"

IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
CSV_EXTS   = {".csv"}
PDF_EXTS   = {".pdf"}  # Skipped

AI_TABLE  = "invoices_ai"
CSV_TABLE = "invoices_csv"

# Canonical schema used by both tables
CANONICAL_COLS = [
    "invoice_number",
    "invoice_date",
    "supplier_name",
    "customer_name",
    "subtotal",
    "tax",
    "total",
    "currency",
    "source_file",
    "source_type"
]


# ============================================================
# 3. CSV → Canonical Field Mapping (based on YOUR FILE)
# ============================================================
CSV_COLUMN_MAP = {
    "invoice_number": "InvoiceID",
    "invoice_date":   "InvoiceDate",
    "supplier_name":  "VendorName",
    "customer_name":  None,       # Not present in CSV
    "subtotal":       "Amount",   # CSV has only Amount
    "tax":            None,       # No tax field
    "total":          "Amount",   # Same as subtotal
    "currency":       "Currency"
}


# ============================================================
# 4. AI Vision Extraction Function (Images Only)
# ============================================================
def extract_invoice_ai(image_path: str) -> dict:
    with open(image_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("ascii")

    payload = {
        "messages": [
            {
                "role": "system",
                "content": [{
                    "type": "text",
                    "text": (
                        "You extract structured data from invoices. "
                        "Return ONLY valid JSON."
                    )
                }]
            },
            {
                "role": "user",
                "content": [
                    {"type": "image_url", "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded}"
                    }},
                    {"type": "text", "text": (
                        "Extract JSON:\n"
                        "{"
                        "\"invoice_number\":string,"
                        "\"invoice_date\":string,"
                        "\"supplier_name\":string,"
                        "\"customer_name\":string,"
                        "\"subtotal\":number|null,"
                        "\"tax\":number|null,"
                        "\"total\":number|null,"
                        "\"currency\":string|null"
                        "}"
                    )}
                ]
            }
        ],
        "temperature": 0.0,
        "max_tokens": 300
    }

    resp = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    resp.raise_for_status()

    content = resp.json()["choices"][0]["message"]["content"].strip()

    # Remove wrapping ```
    if content.startswith("```"):
        content = content.strip("`")
        if content.startswith("json"):
            content = content[4:].strip()

    return json.loads(content)


# ============================================================
# 5. CSV Loader Function (Pandas → Canonical)
# ============================================================
def load_csv_file(csv_path: str, filename: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # Add canonical columns
    for col in CANONICAL_COLS:
        if col not in df.columns:
            df[col] = None

    # Apply mapping
    for canon_col, csv_col in CSV_COLUMN_MAP.items():
        if csv_col and csv_col in df.columns:
            df[canon_col] = df[csv_col]

    df["source_file"] = filename
    df["source_type"] = "csv"

    return df[CANONICAL_COLS]


# ============================================================
# 6. Main ETL: Run AI + CSV Pipelines
# ============================================================
ai_rows = []
csv_frames = []

print(f"Scanning folder: {FOLDER_PATH}")

for filename in os.listdir(FOLDER_PATH):
    full_path = os.path.join(FOLDER_PATH, filename)

    if not os.path.isfile(full_path):
        continue

    _, ext = os.path.splitext(filename)
    ext = ext.lower()

    # --- IMAGE → AI ---
    if ext in IMAGE_EXTS:
        try:
            print(f"🖼 AI processing: {filename}")
            data = extract_invoice_ai(full_path)

            for c in CANONICAL_COLS:
                data.setdefault(c, None)

            data["source_file"] = filename
            data["source_type"] = "image-ai"
            ai_rows.append(data)
            print(f"✔ AI done: {filename}")
        except Exception as e:
            print(f"❌ AI error {filename}: {e}")
        continue

    # --- CSV → Direct Load ---
    if ext in CSV_EXTS:
        try:
            print(f"📄 CSV loading: {filename}")
            pdf = load_csv_file(full_path, filename)
            csv_frames.append(pdf)
            print(f"✔ CSV done: {filename}")
        except Exception as e:
            print(f"❌ CSV error {filename}: {e}")
        continue

    # --- Skip PDFs ---
    if ext in PDF_EXTS:
        print(f"⏭ Skipping PDF: {filename}")
        continue

    print(f"Skipping unsupported file: {filename}")


# ============================================================
# 7. Save Each Pipeline to Its Own Table
# ============================================================

# AI TABLE
if ai_rows:
    df_ai = spark.createDataFrame(pd.DataFrame(ai_rows))
    df_ai.write.mode("overwrite").saveAsTable(AI_TABLE)
    print(f"✅ Saved AI data → {AI_TABLE}")
    df_ai.show(10, truncate=False)
else:
    print("⚠ No AI image rows found.")

# CSV TABLE
if csv_frames:
    df_csv = spark.createDataFrame(pd.concat(csv_frames, ignore_index=True))

    (df_csv.write
          .mode("overwrite")
          .option("overwriteSchema", "true")   # 👈 important
          .saveAsTable(CSV_TABLE))

    print(f"✅ Saved CSV data → {CSV_TABLE}")
    df_csv.show(10, truncate=False)
else:
    print("⚠ No CSV rows found.")



StatementMeta(, b36530e9-47ee-4629-8237-129dd2687f6e, 3, Finished, Available, Finished)

Scanning folder: /lakehouse/default/Files/rawdocs
🖼 AI processing: invoice_1.jpg
✔ AI done: invoice_1.jpg
⏭ Skipping PDF: invoice_20.pdf
📄 CSV loading: invoices_master.csv
✔ CSV done: invoices_master.csv
✅ Saved AI data → invoices_ai
+--------------+------------+-------------------+-----------------+--------+----+------+--------+-------------+-----------+
|invoice_number|invoice_date|supplier_name      |customer_name    |subtotal|tax |total |currency|source_file  |source_type|
+--------------+------------+-------------------+-----------------+--------+----+------+--------+-------------+-----------+
|83457         |Sept. 6     |Clearwave Logistics|Everfresh Markets|NULL    |NULL|3940.0|USD     |invoice_1.jpg|image-ai   |
+--------------+------------+-------------------+-----------------+--------+----+------+--------+-------------+-----------+

✅ Saved CSV data → invoices_csv
+--------------+------------+--------------------+-------------+--------+----+-------+--------+------------------

In [1]:
spark.sql("SHOW TABLES").show()


StatementMeta(, 95b59dce-55da-40a2-9783-cfeca9314de7, 3, Finished, Available, Finished)

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| invoices|     invoices_master|      false|
| invoices|ai_extracted_invo...|      false|
| invoices|      invoices_final|      false|
| invoices|         invoices_ai|      false|
| invoices|        invoices_csv|      false|
| invoices|       invoices_gold|      false|
+---------+--------------------+-----------+

